In [ ]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import*
from sys import stdout

import time

In [ ]:
start_time = time.time()

prmtop = AmberPrmtopFile('/home/eva/Documents/structures/explicit-fromh++/ff19SB-OPC/hex_L24E/pH6/6q5j_pH6_explicit.top')
inpcrd = AmberInpcrdFile('/home/eva/Documents/structures/explicit-fromh++/ff19SB-OPC/hex_L24E/pH6/6q5j_pH6_explicit.crd')

system = prmtop.createSystem(nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds, switchDistance=0.9*nanometers)

#include barostat to run simulation in constant pressure
system.addForce(MonteCarloBarostat(1*bar, 298.15*kelvin))

#Langevin Integrators for constant temperature
integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.002*picoseconds)


platform = Platform.getPlatformByName('CUDA')

properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('output.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.reporters.append(CheckpointReporter('checkpnt.chk', 5000))
simulation.step(100000000)

print ("simulation time:", time.time() - start_time, "s")